In [1]:
import shutil

from core.gain import *
from core.rnn_predic import *
from core.models import *
from core.util import *
#from core.window import WindowGenerator, MissData, make_dataset_water, WaterDataGenerator
from core.window import WindowGenerator, make_dataset_gain, make_dataset_water
from core.file_open import make_dataframe
from core.miss_data import MissData
import json
%matplotlib widget
import os

In [2]:
def hour_to_day_mean(array):
    time = 24
    array = array.reshape((array.shape[0], array.shape[1] // time, time, array.shape[2]))
    array = array.mean(2)
    return array

In [3]:

def compa(model=None,df = None, plot_col=0, input_width=7*24, label_width=5*24, target_std=None, target_mean=None, predict_day=4):
    width = input_width + label_width
    length = df.shape[0] - width
    inputs = []
    labels = []
    
    for i in range(0, length, 24):
        dataset = df.iloc[i:i+width].to_numpy()
        input = dataset[:input_width]
        label = dataset[input_width:, plot_col:plot_col+1]
        
        input = input.reshape((-1,)+input.shape)
        label = label.reshape((-1,)+label.shape)
        
        inputs.append(input)
        labels.append(label)
        
        
    inputs = np.concatenate(inputs, axis=0)
    labels = np.concatenate(labels, axis=0)
    

    predictions = model(inputs).numpy()
    
    predictions = predictions * target_std[plot_col] + target_mean[plot_col]
    labels = labels * target_std[plot_col] + target_mean[plot_col]

    pred_day = hour_to_day_mean(predictions)
    
    label_day = hour_to_day_mean(labels)
    
    inputs_target = inputs[:,:,plot_col:plot_col+1]
    inputs_target = inputs_target * target_std[plot_col] + target_mean[plot_col]
    inputs_day = hour_to_day_mean(inputs_target)
    
    plt.figure()
    #input_index = np.array(range(0, length, 24))
    input_index = np.array(range(inputs_day.shape[0]))
    #label_index = input_index + 24* (7 + predict_day)
    label_index = input_index + (7 + predict_day)
    plt.plot(input_index, inputs_day[:, 0, :], label='input')
    plt.plot(label_index, label_day[:, predict_day, :], label='label')
    plt.plot(label_index, pred_day[:, predict_day, :], label='pred')
    plt.legend()
    #plt.savefig('test_plt.png')
    
    o1 = np.mean(label_day[:,predict_day,:])
    nse1 = ((label_day - pred_day)**2).sum(axis=0)
    nse2 = ((label_day - o1)**2).sum(axis=0)
    nse3 = 1 - (nse1[predict_day]/nse2[predict_day])
    pbias1 = (label_day - pred_day).sum(axis=0)
    pbias2 = (label_day).sum(axis=0)
    pbias3 = (pbias1[predict_day]/pbias2[predict_day])*100
    
    
    labels_test = labels.mean(axis=1)
    predis_test = inputs_target.mean(axis=1)
    
    nse2_1 = ((labels_test - predis_test)**2).sum()
    nse2_2 = ((labels_test - o1)**2).sum()
    nse2_3 = 1 - (nse2_1/nse2_2)
    
    pbias2_1 = (labels_test - predis_test).sum()
    pbias2_2 = labels_test.sum()
    pbias2_3 = pbias2_1/pbias2_2 * 100
    
    mae = (np.abs(label_day - pred_day)).mean()
    mse = ((label_day - pred_day)**2).mean()
    rmse = np.sqrt(((label_day - pred_day)**2).mean())
    
    o_ = np.mean(label_day[:,predict_day,:])
    p_ = np.mean(pred_day[:,predict_day,:])
    
    oi = label_day[:,predict_day,:]
    pi = pred_day[:,predict_day,:]
    
    high = ((oi-o_)*(pi-p_)).sum()
    low = np.sqrt( ( (oi-o_)**2 ).sum() )
    low = low * np.sqrt( ( (pi-p_)**2 ).sum() )
    
    R = high/low
    RS = (R)**2
    
    return nse3, pbias3, pred_day, labels, mae, rmse, RS, R

In [4]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e) 


1 Physical GPUs, 1 Logical GPUs


In [5]:
data_path = 'data/'
parameters_dir = 'input'

parameters_file = 'input.json'
parameters_path = '{dir}/{file}'.format(dir=parameters_dir, file=parameters_file)

with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

gain_parameters = parameters['gain']
rnn_parameters = parameters['rnn']
file_parameters = parameters['file']

In [6]:
file_parameters['watershed']

'geum'

In [7]:
parameters_path = parameters_dir+'/'+ file_parameters['watershed'] + '.json'
with open(parameters_path, encoding='utf8') as json_file:
    parameters = json.load(json_file)

In [8]:
pd.set_option('display.max_columns', 1000)

In [9]:
data_parameters = parameters['data']

interpolation_option = data_parameters['interpolation']
colum_idx = data_parameters['columns']
watershed = data_parameters['watershed']
file_names = data_parameters['files']
folder = data_parameters['directorys']
for i in range(len(folder)):
    folder[i] = watershed+folder[i]

In [10]:
__GAIN_TRAINING__ = gain_parameters['train']
gain_epochs = gain_parameters['max_epochs']
gain_in_setps = gain_parameters['input_width']
gain_out_setps = gain_parameters['label_width']
gain_batch_size = gain_parameters['batch_size']
gain_fill_no = gain_parameters['fill_width']
gain_shift = gain_parameters['shift_width']
gain_miss_rate = gain_parameters['miss_rate']

__RNN_TRAINING__ = rnn_parameters['train']
rnn_epochs = rnn_parameters['max_epochs']
rnn_in_setps = rnn_parameters['input_width']
rnn_out_steps = rnn_parameters['label_width']
rnn_batch_size = rnn_parameters['batch_size']
rnn_predict_day = rnn_parameters['predict_day']
rnn_target_column = rnn_parameters['target_column']

if rnn_predict_day < 3 or rnn_predict_day >5:
    print('predict_day err')
    exit(88)
rnn_predict_day -= 1

In [11]:
__GAIN_TRAINING__, gain_epochs ,__RNN_TRAINING__, rnn_epochs

(True, 2000, False, 15)

In [12]:

if file_parameters['watershed'] == 'nak':
    run_num = range(len(folder))
else:
    run_num = [0]



real_df_all = pd.DataFrame([])
target_all = target_mean = target_std = 0

gain_val_performance = {}
gain_performance = {}

length = len(run_num)

ddday = 31
mmmonth = 12

#length = 3

In [13]:
for i in range(length):

    idx = run_num[i]

    print('interpol flag : ', interpolation_option[idx])
    print('folder : ', data_path + folder[idx])
    print('colum_idx : ', colum_idx[idx])
    print('file_names[idx] : ', file_names[idx])

    #df, times = make_dataframe(data_path+folder[idx], file_names[idx], colum_idx[idx], interpolation=interpolation_option[idx])
    df, times, mmmonth, ddday = make_dataframe(data_path+folder[idx], file_names[idx], 
                               colum_idx[idx], interpolation=interpolation_option[idx],
                              first_file_no=i, month=mmmonth, day=ddday)

    df_all, train_mean, train_std, df = normalize(df)


    if i == 0:
        dfff = df
        target_all = df_all
        target_std = train_std
        target_mean = train_mean
        start_year = str(times.iloc[0].year)
        end_year = str(times.iloc[-1].year)

    if interpolation_option[idx][0] == False:

        loadfiles = ['idx.npy', 'miss.npy', 'discriminator.h5', 'generator.h5']

        gain_calc_falg = True

        if __GAIN_TRAINING__ == True:
            gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
            #print(folder[idx], ': training ', 'Miss date save : ', gain_calc_falg)
        else:
            for file in loadfiles:
                if os.path.isfile('save/' + folder[idx]+file):
                    shutil.copyfile('save/' + folder[idx]+file, 'save/'+file)
                    #print('load file name : save/' + folder[idx]+file)
                else:
                    if file == 'miss.npy':
                        gain_calc_falg = MissData.save(pd.concat(df, axis=0).to_numpy(), max_tseq=24, save_dir='save/')
                        #print(folder[idx], ': is not miss.npy ', 'Miss date save : ', gain_calc_falg)

        if gain_calc_falg == True:
            #print('GainWindowGenerator in main')
            WindowGenerator.make_dataset = make_dataset_gain
            wide_window = WindowGenerator(input_width=gain_in_setps, label_width=gain_out_setps, shift=gain_shift,
                                          fill_no=gain_fill_no, miss_rate=gain_miss_rate, batch_size=gain_batch_size,
                                          train_df = df_all, val_df = df_all, test_df = df_all, df = df)

            #gain = model_GAIN(shape=wide_window.dg.shape[1:], gen_sigmoid=False, epochs=gain_epochs, training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')
            gain = model_GAIN(shape=(gain_in_setps, df_all.shape[1]), gen_sigmoid=False, epochs=gain_epochs,
                              training_flag=__GAIN_TRAINING__, window=wide_window, model_save_path='save/')

            gain_val_performance[str(i)] = gain.evaluate(wide_window.val)
            gain_performance[str(i)] = gain.evaluate(wide_window.test, verbose=0)

            #print('file proc in main')
            if __GAIN_TRAINING__ == True:
                #dir = 'save/'+folder[i]
                if not os.path.exists('save/' + folder[idx]):
                    os.makedirs('save/'+folder[idx])
                for file in loadfiles:
                    shutil.copyfile('save/' + file, 'save/' + folder[idx] + file)

            #print('create_dataset_with_gain in main')
            #ori, gan = create_dataset_with_gain(gain=gain, window=wide_window, df=df)
            ori, gan = create_dataset_with_gain(gain=gain, shape=(gain_in_setps, df_all.shape[1]), df=df)

        else:
            gan = create_dataset_interpol(window=gain_in_setps, df=df)
    else:
        gan = create_dataset_interpol(window=gain_in_setps, df=df)

    if i == 0 :
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.DataFrame(gan)
    else:
#        if i < length -1:
#            gan = gan[:,:-4]  #맨마지막전까지 사인코사인삭제
#            print(gan.shape)
        real_df_all = pd.concat([real_df_all, pd.DataFrame(gan)], axis=1)

interpol flag :  [False, False]
folder :  data/geum/자동/
colum_idx :  :,[26,27,28,29,30,31,32,33]
file_names[idx] :  [['현도_2013.xlsx', '현도_2014.xlsx', '현도_2015.xlsx', '현도_2016.xlsx', '현도_2017.xlsx', '현도_2018.xlsx', '현도_2019.xlsx', '현도_2020.xlsx'], ['대청호_2013.xlsx', '대청호_2014.xlsx', '대청호_2015.xlsx', '대청호_2016.xlsx', '대청호_2017.xlsx', '대청호_2018.xlsx', '대청호_2019.xlsx', '대청호_2020.xlsx']]
MissData :  save/  miss :  (6828, 12)
MissData :  save/  miss :  (6828, 12)
Epoch 1/2000
1/1 [==============================] - 0s 195ms/step - gen_loss: 117.8875 - disc_loss: 0.7219 - rmse: 1.0436 - val_loss: 0.9389
Epoch 2/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 95.8617 - disc_loss: 0.5315 - rmse: 0.9415 - val_loss: 0.8938
Epoch 3/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 78.7178 - disc_loss: 0.4601 - rmse: 0.8122 - val_loss: 0.8191
Epoch 4/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 72.3045 - disc_loss: 0.4047 - rmse: 0.8185 

1/1 [==============================] - 0s 20ms/step - gen_loss: 7.8929 - disc_loss: 0.3679 - rmse: 0.3515 - val_loss: 0.7327
Epoch 57/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 8.9602 - disc_loss: 0.3525 - rmse: 0.3526 - val_loss: 0.4318
Epoch 58/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.0159 - disc_loss: 0.3335 - rmse: 0.3532 - val_loss: 0.3256
Epoch 59/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 11.0286 - disc_loss: 0.3530 - rmse: 0.3995 - val_loss: 0.3174
Epoch 60/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 29.4411 - disc_loss: 0.3646 - rmse: 0.6545 - val_loss: 0.3374
Epoch 61/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.3064 - disc_loss: 0.3162 - rmse: 0.3365 - val_loss: 0.3478
Epoch 62/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 9.4025 - disc_loss: 0.3260 - rmse: 0.3245 - val_loss: 0.8540
Epoch 63/2000
1/1 [====================

Epoch 115/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.5058 - disc_loss: 0.2872 - rmse: 0.2870 - val_loss: 0.2732
Epoch 116/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.2542 - disc_loss: 0.2854 - rmse: 0.2601 - val_loss: 0.3087
Epoch 117/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.3773 - disc_loss: 0.2653 - rmse: 0.2832 - val_loss: 0.2663
Epoch 118/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.9124 - disc_loss: 0.2743 - rmse: 0.2826 - val_loss: 0.2908
Epoch 119/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.5512 - disc_loss: 0.2655 - rmse: 0.2625 - val_loss: 0.6826
Epoch 120/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 6.8797 - disc_loss: 0.2672 - rmse: 0.2602 - val_loss: 0.4632
Epoch 121/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4655 - disc_loss: 0.2692 - rmse: 0.2971 - val_loss: 0.2387
Epoch 122/2000
1/1 [

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.2020 - disc_loss: 0.2433 - rmse: 0.2393 - val_loss: 0.6314
Epoch 174/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 3.9302 - disc_loss: 0.2542 - rmse: 0.2479 - val_loss: 0.2533
Epoch 175/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1144 - disc_loss: 0.2501 - rmse: 0.2269 - val_loss: 0.4327
Epoch 176/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.5493 - disc_loss: 0.2618 - rmse: 0.2457 - val_loss: 0.2360
Epoch 177/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4507 - disc_loss: 0.2585 - rmse: 0.2549 - val_loss: 0.2707
Epoch 178/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0439 - disc_loss: 0.2542 - rmse: 0.2771 - val_loss: 0.6077
Epoch 179/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 32.7915 - disc_loss: 0.2394 - rmse: 0.7904 - val_loss: 0.2957
Epoch 180/2000
1/1 [==============

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.3547 - disc_loss: 0.2182 - rmse: 0.2515 - val_loss: 0.3453
Epoch 232/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.8123 - disc_loss: 0.2338 - rmse: 0.2823 - val_loss: 0.2243
Epoch 233/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 21.8516 - disc_loss: 0.2383 - rmse: 0.6805 - val_loss: 0.2233
Epoch 234/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 27.5461 - disc_loss: 0.2360 - rmse: 0.7484 - val_loss: 0.2704
Epoch 235/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 4.4257 - disc_loss: 0.2323 - rmse: 0.2512 - val_loss: 0.2324
Epoch 236/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.6409 - disc_loss: 0.2252 - rmse: 0.2154 - val_loss: 0.2152
Epoch 237/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.0027 - disc_loss: 0.2411 - rmse: 0.2800 - val_loss: 0.2503
Epoch 238/2000
1/1 [=============

...

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.6402 - disc_loss: 0.1603 - rmse: 0.2244 - val_loss: 0.2005
Epoch 1450/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.2139 - disc_loss: 0.1728 - rmse: 0.2018 - val_loss: 0.1864
Epoch 1451/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8033 - disc_loss: 0.1661 - rmse: 0.2178 - val_loss: 0.2463
Epoch 1452/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 19.7675 - disc_loss: 0.1666 - rmse: 0.5236 - val_loss: 0.2113
Epoch 1453/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0638 - disc_loss: 0.1685 - rmse: 0.2296 - val_loss: 0.2115
Epoch 1454/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.7537 - disc_loss: 0.1642 - rmse: 0.2076 - val_loss: 0.2149
Epoch 1455/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3890 - disc_loss: 0.1529 - rmse: 0.1786 - val_loss: 0.2359
Epoch 1456/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 4.0831 - disc_loss: 0.1771 - rmse: 0.2494 - val_loss: 0.2151
Epoch 1508/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.1879 - disc_loss: 0.1591 - rmse: 0.2666 - val_loss: 0.2253
Epoch 1509/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.4982 - disc_loss: 0.1618 - rmse: 0.2071 - val_loss: 0.2062
Epoch 1510/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 14.8231 - disc_loss: 0.1604 - rmse: 0.3421 - val_loss: 0.2583
Epoch 1511/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3810 - disc_loss: 0.1575 - rmse: 0.2245 - val_loss: 0.5080
Epoch 1512/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1671 - disc_loss: 0.1494 - rmse: 0.1935 - val_loss: 0.2094
Epoch 1513/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 5.4662 - disc_loss: 0.1612 - rmse: 0.2594 - val_loss: 0.2318
Epoch 1514/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 3.3704 - disc_loss: 0.1661 - rmse: 0.2245 - val_loss: 0.2465
Epoch 1566/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.7089 - disc_loss: 0.1680 - rmse: 0.2568 - val_loss: 0.2480
Epoch 1567/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 5.0158 - disc_loss: 0.1729 - rmse: 0.2630 - val_loss: 0.4236
Epoch 1568/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.9585 - disc_loss: 0.1730 - rmse: 0.2114 - val_loss: 0.2315
Epoch 1569/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 2.7671 - disc_loss: 0.1589 - rmse: 0.1992 - val_loss: 0.3534
Epoch 1570/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.1390 - disc_loss: 0.1625 - rmse: 0.2083 - val_loss: 0.5990
Epoch 1571/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9909 - disc_loss: 0.1484 - rmse: 0.2192 - val_loss: 0.2358
Epoch 1572/2000
1/1 [========

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9392 - disc_loss: 0.1618 - rmse: 0.2081 - val_loss: 0.1908
Epoch 1624/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8435 - disc_loss: 0.1574 - rmse: 0.2004 - val_loss: 0.2079
Epoch 1625/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4447 - disc_loss: 0.1515 - rmse: 0.1859 - val_loss: 0.2025
Epoch 1626/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 17.6091 - disc_loss: 0.1571 - rmse: 0.3626 - val_loss: 0.2615
Epoch 1627/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.7693 - disc_loss: 0.1670 - rmse: 0.2199 - val_loss: 0.2012
Epoch 1628/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.3539 - disc_loss: 0.1616 - rmse: 0.1729 - val_loss: 0.2306
Epoch 1629/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.8359 - disc_loss: 0.1651 - rmse: 0.2779 - val_loss: 0.2466
Epoch 1630/2000
1/1 [=======

1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9370 - disc_loss: 0.1511 - rmse: 0.1934 - val_loss: 0.4578
Epoch 1682/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 3.0575 - disc_loss: 0.1489 - rmse: 0.2041 - val_loss: 0.2435
Epoch 1683/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.9175 - disc_loss: 0.1582 - rmse: 0.2133 - val_loss: 0.2341
Epoch 1684/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 25.9998 - disc_loss: 0.1589 - rmse: 0.5804 - val_loss: 0.2127
Epoch 1685/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 2.4941 - disc_loss: 0.1569 - rmse: 0.2007 - val_loss: 0.2143
Epoch 1686/2000
1/1 [==============================] - 0s 20ms/step - gen_loss: 27.4790 - disc_loss: 0.1705 - rmse: 0.5512 - val_loss: 0.2273
Epoch 1687/2000
1/1 [==============================] - 0s 21ms/step - gen_loss: 4.3026 - disc_loss: 0.1587 - rmse: 0.2284 - val_loss: 0.1803
Epoch 1688/2000
1/1 [======

In [14]:
#-----------------------------------------------------------------------------------------------------------------
#            RRRRRRRRRRRRRRRRRR          NNNNNN       NNNNNN      NNNNNN       NNNNNN
#           RRRRRRRRRRRRRRRRRRRR        NNNNNNN      NNNNNN      NNNNNNN      NNNNNN
#          RRRRRRR      RRRRRRRR       NNNNNNNN     NNNNNN      NNNNNNNN     NNNNNN
#         RRRRRRR      RRRRRRRR       NNNNNNNNN    NNNNNN      NNNNNNNNN    NNNNNN
#        RRRRRRRRRRRRRRRRRRRR        NNNNNNNNNN   NNNNNN      NNNNNNNNNN   NNNNNN
#       RRRRRRRRRRRRRRRRR           NNNNNNNNNNN  NNNNNN      NNNNNNNNNNN  NNNNNN 
#      RRRRRRR     RRRRRRR         NNNNNNN NNNN NNNNNN      NNNNNNN NNNN NNNNNN
#     RRRRRRR       RRRRRRR       NNNNNNN  NNNNNNNNNN      NNNNNNN  NNNNNNNNNN
#    RRRRRRR        RRRRRRR      NNNNNNN   NNNNNNNNN      NNNNNNN   NNNNNNNNN
#   RRRRRRR         RRRRRRR     NNNNNNN    NNNNNNNN      NNNNNNN    NNNNNNNN 
#  RRRRRRR         RRRRRRR     NNNNNNN     NNNNNNN      NNNNNNN     NNNNNNN
#-----------------------------------------------------------------------------------------------------------------

In [15]:

#pd.DataFrame(gan).tail
#real_df_all.tail



In [16]:
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

print("target_col_idx : ", target_col_idx)
print('out_num_features : ', out_num_features)

label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}
target_col_idx :  2
out_num_features :  1


In [35]:
#ctrl f -- 타겟

print(real_df_all.shape)
rnn_target_column = 'tp'
print(rnn_target_column)



(67656, 24)
tp


In [36]:

#train_df, test_df, val_df, test_df2 = dataset_slice(real_df_all, 0.8, 0.1, 0.1)
#teg_check = 'train=8, test=1, val=1'

train_df, test_df, val_df = dataset_slice(real_df_all, 0.7, 0.1, 0.2)
teg_check = 'train=7, val=1, test=2'

print('train_df.shape : ', train_df.shape, 'val_df.shape : ', val_df.shape, 'test_df.shape:' ,test_df.shape)

train_df.shape :  (47359, 24) val_df.shape :  (13532, 24) test_df.shape: (6765, 24)


In [37]:
#fake_df.shape[0]/8760

In [38]:
idx = [2, 4, 5, 6, 7]
pa = ["do/", "toc/", "nitrogen/", "phosphorus/", "chlorophyll-a/"]
indices = {name: i for i, name in enumerate(idx)}
target_dic = {"do":"do_value", "toc":"toc_value", "tn":"총질소_값", "tp":"총인_값", "chl-a":"클로로필-a_값"}
label_columns_indices = {name: i for i, name in enumerate(dfff[0])}
print("label_columns_indices:")
print(label_columns_indices)


label_columns_indices:
{'tmpr_value': 0, 'ph_value': 1, 'do_value': 2, 'ec_value': 3, 'toc_value': 4, '총질소_값': 5, '총인_값': 6, '클로로필-a_값': 7, 'Day sin': 8, 'Day cos': 9, 'Year sin': 10, 'Year cos': 11}


In [39]:
target_col_idx = label_columns_indices[target_dic[rnn_target_column]]
out_features = [target_col_idx]
out_num_features = len(out_features)

In [40]:
WindowGenerator.make_dataset = make_dataset_water
multi_window = WindowGenerator(
    input_width=rnn_in_setps,label_width=rnn_out_steps, shift=rnn_out_steps,out_features=out_features,
    out_num_features=out_num_features,label_columns=dfff[0].columns, batch_size=rnn_batch_size,
    train_df=train_df, val_df=val_df, test_df=test_df)

model_path = "save/" + watershed + "models/" + file_parameters['watershed'] + '/'+  pa[indices[target_col_idx]]
print("save model path : ", model_path)

if __RNN_TRAINING__:
    if not os.path.exists(model_path):
        os.makedirs(model_path)

save model path :  save/geum/models/geum/phosphorus/


In [41]:
rnn_epochs, rnn_in_setps

(15, 240)

## 모델 학습

In [42]:
gru_model = model_gru(
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
    window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 5,
    #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=2, steps_per_epoch = 2,
    #training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    training_flag=True, checkpoint_path=model_path+"gru.ckpt", continue_train=False)

Epoch 1/15
5/5 [==============================] - 9s 2s/step - loss: 0.6711 - mean_absolute_error: 0.4906 - nse: 0.0138 - val_loss: 1.2380 - val_mean_absolute_error: 0.7110 - val_nse: 0.2279
Epoch 2/15
5/5 [==============================] - 9s 2s/step - loss: 0.4971 - mean_absolute_error: 0.4166 - nse: 0.3451 - val_loss: 0.8009 - val_mean_absolute_error: 0.5532 - val_nse: 0.4975
Epoch 3/15
5/5 [==============================] - 9s 2s/step - loss: 0.4439 - mean_absolute_error: 0.3841 - nse: 0.5727 - val_loss: 0.5032 - val_mean_absolute_error: 0.4355 - val_nse: 0.6793
Epoch 4/15
5/5 [==============================] - 9s 2s/step - loss: 0.2750 - mean_absolute_error: 0.3217 - nse: 0.6598 - val_loss: 0.4171 - val_mean_absolute_error: 0.3691 - val_nse: 0.7354
Epoch 5/15
5/5 [==============================] - 9s 2s/step - loss: 0.2660 - mean_absolute_error: 0.3011 - nse: 0.7544 - val_loss: 0.3293 - val_mean_absolute_error: 0.3531 - val_nse: 0.7858
Epoch 6/15
5/5 [=============================

In [43]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=real_df_all, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)
    
print(model_path)
print("year : " + start_year + " ~ "+ end_year)
print('run : ', run_num)
print('target : ', rnn_target_column , " ", out_features[0])
print("length : ",real_df_all.shape[0])
print(teg_check)

print('all_nse : ', val_nse['GRU'])
print('all_pbias : ', val_pbias['GRU'])
print('all_mae : ', _mae)
print('all_rmse : ', _rmse)
print('all_Rs : ', _Rs)
print('all_R : ', _R)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

save/geum/models/geum/phosphorus/
year : 2013 ~ 2020
run :  [0]
target :  tp   6
length :  67656
train=7, val=1, test=2
all_nse :  [0.809512]
all_pbias :  [-1.3874009]
all_mae :  0.0020254932
all_rmse :  0.0037053823
all_Rs :  0.8097233913100013
all_R :  0.8998463


In [44]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=train_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('train_nse : ', val_nse['GRU'])
print('train_pbias : ', val_pbias['GRU'])
print('train_mae : ', _mae)
print('train_rmse : ', _rmse)
print('train_Rs : ', _Rs)
print('train_R : ', _R)

#print('nse_train : ', val_nse['GRU'])

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
train_nse :  [0.8423238]
train_pbias :  [-1.7716938]
train_mae :  0.0015760325
train_rmse :  0.0028395306
train_Rs :  0.843415296145622
train_R :  0.91837645


In [45]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=test_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)
    #predict_day = 4)

print('----------------------------')
print('val_nse : ', val_nse['GRU'])
print('val_pbias : ', val_pbias['GRU'])
print('val_mae : ', _mae)
print('val_rmse : ', _rmse)
print('val_Rs : ', _Rs)
print('val_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
val_nse :  [0.8353059]
val_pbias :  [-2.747975]
val_mae :  0.003368974
val_rmse :  0.004909482
val_Rs :  0.8364629861547179
val_R :  0.9145835


In [46]:
val_nse = {}
val_pbias = {}
val_nse['GRU'], val_pbias['GRU'], pred, label, _mae, _rmse, _Rs, _R = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    predict_day = rnn_predict_day)

print('----------------------------')
print('test_nse : ', val_nse['GRU'])
print('test_pbias : ', val_pbias['GRU'])
print('test_mae : ', _mae)
print('test_rmse : ', _rmse)
print('test_Rs : ', _Rs)
print('test_R : ', _R)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

----------------------------
test_nse :  [0.7422116]
test_pbias :  [-3.2433822]
test_mae :  0.003006271
test_rmse :  0.005115901
test_Rs :  0.7436445189299299
test_R :  0.86234826


In [29]:
#train_df.shape, val_df.shape, test_df.shape

In [30]:
#-----------------------------------------------------------------------------------------------------------------
# 123SFACVTHYI78567UDBYUJM534523RDHGTHETYIE5I8I78O9P789J467UBDFGDASFSADCXVFDSWERTNERGRTTUTYAFGASDFSDBGFDF67E54563
# 64534FDSFVSDHT56SDFG656USRHSDYA3Y RTA WERGERGSDFGVTY34TASDF dfasdfghhgbcvbsdfgrwergfdsfvzxcvasdfaweqdqasdfderadf
# feasetttttttttttttttttttvxzcvxzvxcvxvnjfghjvb111nfghjdfhgdhthrtyyyyyyyyyyrtjliuioyuytrtyurtyughjuyityuityuityuijjty
# hgfgdrrtyetghfdghvcvbnhjfghjgggnfnmjhkuytlryewqwerasdfgghrthfgjyruhjyuihjknhybgtvfrcdezxcabnghrtyreeghghte45654y  
# tr345rgbbbdfghterty4534562345jhgy5462345234534n5345g345t3245d34f235345f345g34454f345t345er345234yt545y45tg45t534
#       SSSSSSSSSSSSSS         ASDFALKJIVAJLMSEFLKMALSKVJALKSRKLTGAKLSDGVLK   *****    ***************   
#            SSSSSSSSSSSSS               ******            ******              *****    *************       (*_*)  
#                   SSSSSSSS               ******            ******              *****    *****            ~( (>
#        SSSS         S  SSS               ******            ******              *****    *****            / (          
#      SSSS          S  SSSS               ******            ******              *****    *****                     
#     SSSSS            SSSS                ******             ******            ******    *****                     
#      SSSSSSSSSSSSSSSSSS                  ******              *********************      *****                   
#-----------------------------------------------------------------------------------------------------------------
stop = stop

NameError: name 'stop' is not defined

In [ ]:
'''
sub_flag = True
history_nse = []

bob = 0
b = 1

while sub_flag:
    

    
    gru_model = model_gru(
        #window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=rnn_epochs,
        window=multi_window, OUT_STEPS=rnn_out_steps, out_num_features=out_num_features, epochs=15, steps_per_epoch = 4,
        training_flag=__RNN_TRAINING__, checkpoint_path=model_path+"gru.ckpt", continue_train=False)
    
    #nse, _, _, _ = compa(
        #model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
        #predict_day = rnn_predict_day)
        #predict_day = 4)
    
    nse, _, _, _, _, _, _, _ = compa(
    model=gru_model,df=val_df, plot_col=out_features[0], target_std=target_std, target_mean=target_mean,
    #predict_day = rnn_predict_day)
    predict_day = 4)
    
    if(nse > 0.5):
        sub_flag = False
   
    
    history_nse.append(nse)
    
    print('--------------------------')
    print(history_nse)
    print(nse)
    print('--------------------------')
    
        
    bob += b
    if(bob < -4):
        b = 1
    if(bob > 0):
        b = -1
'''
